In [1]:
!pip install docplex
import docplex.mp
import sys
import docplex.cp
from docplex.cp.model import *
from docplex.mp.model import *

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
model = Model(name='cs170')

file = open('10.in', 'r').read().split()

size = int(file[0])

stress_max = float(file[1])

stress = {}    
happiness = {}


for i in range(2, len(file)):
    if i % 4 == 2:
        first = int(file[i])
        second = int(file[i+1])
        happiness[(first, second)] = float(file[i+2])
        stress[(first, second)] = float(file[i+3])
        
# x_roomi, person1, person2 --> binary variables
x = {(room, person1, person2): model.binary_var(name='x_{0}_{1}_{2}'.format(room, person1, person2)) for room in range(size) for (person1, person2) in happiness}



stress_rooms = {room: model.continuous_var(name = 'stress_{0}'.format(room)) for room in range(size)}
happiness_rooms = {room: model.continuous_var(name = 'happiness_{0}'.format(room)) for room in range(size)}
room_is_filled = {room: model.binary_var(name = 'filled_{0}'.format(room)) for room in range(size)}
k = model.integer_var(name = 'k')

# number of rooms filled = k
model.add_constraint(k == model.sum(room_is_filled[i] for i in range(size)))

# R_i constraints
for i in range(size):
    for (j, k) in happiness:
        model.add_constraint(room_is_filled[i] >= x[(i, j, k)])

    
        
for i in range(size):
    model.add_constraint(k * stress_rooms[i] <= stress_max, 'quad')

# happiness and stress

for i in range(size):
    for (j, k) in happiness:
        model.add_constraint(happiness_rooms[i] == model.sum(x[(i, j, k)]*happiness[(j, k)]))
        model.add_constraint(stress_rooms[i] == model.sum(x[(i, j, k)]*stress[(j, k)]))

# only 1 pair in a room

for (j, k) in happiness:
    model.add_constraint(model.sum(x[(i, j, k)] for i in range(size)) <= 1)

    
model.maximize(model.sum(happiness_rooms[i] for i in range(size)))

In [3]:
model.print_information()



Model: cs170
 - number of variables: 481
   - binary=460, integer=1, continuous=20
 - number of constraints: 1406
   - linear=1406
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP


In [4]:
solution = model.solve()
assert solution
solution.display()

DOcplexLimitsExceeded: **** Promotional version. Problem size limits exceeded, CPLEX code=1016

In [5]:
from platform import python_version

print(python_version())


3.7.3
